<h1><center>Bitcoin & The Day of The Week Effect</center></h1>
<center>IESEG 2018 - PIPARO Paolo, MAZZON Filippo, FERRARI Matteo</center></center>
<center>Originally produced by David Yechiam Aharona, Mahmoud Qadan, 2018</center>


# Table of Contents

<a id="contents"></a><span style="color:lightgreen">
1. [Dataset Creation](#first) 
2. [Relationship of BTC price & Google Search Engine](#second) 
3. [Descriptive Statistics](#aa) 
4. [Unit Root Tests | ADF, PP & KPSS](#fourth) 
5. [Dummy Variable Introduction](#fifth) 
6. [OLS Regressions](#six) 
    - [Period 1: Sample A](#7) 
    - [Period 1: Sample B](#8) 
    - [Period 2: Sample A](#9) 
    - [Period 2: Sample B](#10) 
    - [Period 3: Sample A](#11) 
    - [Period 3: Sample B](#12) 

    

In [220]:
#Import all libraries 
import math 
from datetime import datetime
import random
from math import pi 
import operator
from math import e
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

import quandl
quandl.ApiConfig.api_key = 'SMm_kYWFxu9y8B2kBpSr'

import scipy
from scipy import stats
from scipy.stats import norm
from scipy.stats import skew
from scipy.stats import kurtosis
from arch.unitroot import KPSS
from arch.unitroot import PhillipsPerron
from statsmodels.tsa import api as tsa
import statsmodels.api as sm

import dbnomics
from dbnomics import fetch_series

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_iris 
from sklearn import tree

import urllib
import pyorient
import urllib.request
import csv
import feedparser as fp
import json
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime
from termcolor import colored

import pytrends
from pytrends.request import TrendReq
from pytrends.request import TrendReq

import nltk, bs4
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pylab as pl
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import cufflinks as cf  
import configparser as cp

import eikon as ek
ek.set_app_key('482f2d12dd20480298fe43b0d0d6614f98d1f6c5')

This study aims to investigate whether Bitcoin price distribution varies across weekdays. 
If Bitcoin becomes a currency that can be used as a medium of exchange, an alternative to hedging tools or a store of value such as gold, understanding the dynamics and behavior of such a relatively new fin-tech tool is crucial. 
The day of the week effect has been largely studied among scholars, namely related to equities, bonds, commodities and currency, with the exception of BTC. 
Therefore it is interesting to demonstrate if the day-of-the-week effect is present in both the returns and volatility of Bitcoin. This effect has evolved along with the gradual recognition of Bitcoin by market participants. The conclusion is that Mondays are associated with higher returns and volatility in Bitcoin prices compared with other weekdays.

Our data consist of daily observations from October 2010 to October 2017

## Dataset Creation 
<a id="first"></a>
[Index](#contents)

In [2]:
#Time & Date of last output analysis 
first_var= ["BTC="]
time, err = ek.get_data(first_var, ['CF_TIME', 'CF_DATE'])
time, err = ek.get_data(first_var, ['CF_TIME', 'CF_DATE'])
time = time.rename(columns={'CF_TIME': 'Time of Analysis', 'CF_DATE': 'Date of Analysis', })
print('The time of the last analysis was {} and the date was {}'
      .format(time['Time of Analysis'][0], time['Date of Analysis'][0]))

The time of the last analysis was 09:21:38 and the date was 2019-11-24


In [3]:
#Some BTC general info
data_BTC, err = ek.get_data(first_var, ['CF_NAME', 
                                     'TR.AssetCategory', 
                                     'TR.InstrumentDescription',
                                     'CF_BID', 
                                     'CF_ASK', 
                                     'CF_TICK', 
                                     'CF_SOURCE'])
data_BTC = data_BTC.rename(columns={'CF_NAME': 'Asset Name', 
                                    'CF_BID': 'Bid',
                                    'CF_ASK': 'Ask', 
                                    'CF_TICK': 'Last Indicator',
                                    'CF_SOURCE': 'Data Source'  })

data_BTC

,Instrument,Asset Name,Asset Category Description,Instrument Description,Bid,Ask,Last Indicator,Data Source
0,BTC=,Bitcoin,FX Spot Rate,Bitcoin/US Dollar FX Spot Rate,7228.1,7229,⇩,Crypto Comp


In [4]:
#Retrieve BTC OHLC and plot the close-price time series 
#Unfortunately there is not BTC price in Reuters before 2014
df_closeprice = ek.get_timeseries(first_var, 
                                  start_date='2010-10-01', 
                                  end_date='2017-10-01', 
                                  interval='daily')
df_closeprice['BTC Arithm Returns'] = df_closeprice[('CLOSE')].pct_change()
df_closeprice['BTC LN Returns'] = np.log(df_closeprice[('CLOSE')]).diff()  #as used by the original authors
del(df_closeprice['COUNT'])

df_closeprice.head(4)

BTC=,CLOSE,HIGH,LOW,OPEN,BTC Arithm Returns,BTC LN Returns
Date,,,,,,
2014-07-17,623.01,628.94,610.00,613.03,NaN,NaN
2014-07-18,629.70,632.80,618.46,622.73,0.010738,0.010681
2014-07-19,627.53,632.06,624.95,629.71,-0.003446,-0.003452
2014-07-20,624.66,628.89,619.99,628.88,-0.004573,-0.004584


In [5]:
#Try to retrieve BTC data from QUANDL to see if there's the entire period. 
#Unfortunately, on QUANDL the dataset starts in 2014 as well
data_BTC = quandl.get('BITFINEX/BTCUSD', start_date='2010-10-01', end_date='2017-01-01')
data_BTC.head()

,High,Low,Mid,Last,Bid,Ask,Volume
Date,,,,,,,
2014-04-15,513.9000,452.00,504.23500,505.0000,503.5000,504.97,21013.584774
2014-04-16,547.0000,495.00,537.50000,538.0000,537.0000,538.00,29633.358705
2014-04-17,538.5000,486.10,507.02000,508.0000,506.0400,508.00,20709.783819
2014-04-18,509.0000,474.25,483.77000,482.7500,482.7500,484.79,10458.045243
2014-04-19,513.9899,473.83,505.01065,507.4999,502.5313,507.49,8963.618369


In [6]:
#ACTUAL
#Retrieve BTC data from Coin Metric (https://coinmetrics.io/community-data-dictionary/)
#Processed & Cleaned
BTC_file = "/Users/Paolo/Desktop/BTC.xlsx"
BTC_df = pd.read_excel(BTC_file)
#BTC_df.tail()
#BTC_df.head()
type(BTC_df['BTC USD Price'][2])

numpy.float64

In [7]:
BTC_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2591 entries, 0 to 2590
Data columns (total 2 columns):
Date             2591 non-null datetime64[ns]
BTC USD Price    2591 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 40.6 KB


In [8]:
BTC_df.describe()

,BTC USD Price
count,2591.000000
mean,517.408752
std,930.563566
min,0.061065
25%,10.994984
50%,244.383473
75%,589.740954
max,7043.777939


In [9]:
BTC_df['BTC LN Returns'] = round(np.log(BTC_df['BTC USD Price']).diff(),4)
BTC_df = BTC_df.set_index('Date')
BTC_df.head()

,BTC USD Price,BTC LN Returns
Date,,
2010-09-30,0.061900,NaN
2010-10-01,0.061973,0.0012
2010-10-02,0.061400,-0.0093
2010-10-03,0.061065,-0.0055
2010-10-04,0.061294,0.0038


In [11]:
#plot the returns and closing price (with different scaling) for the BTC dataset available

daily = BTC_df['BTC LN Returns'].cumsum().resample('D', label='right').last().ffill()
dailyframe = daily.to_frame()
closing = BTC_df['BTC USD Price']
closingframe = closing.to_frame()
closingframe.join(dailyframe).iplot(secondary_y='BTC LN Returns', width=2.5)

In [12]:
#Retrieve data for Trade Weighted U.S. Dollar Index from Quandl
data_EX = quandl.get('FRED/DTWEXB', start_date='2010-09-30', end_date='2017-11-02')
data_EX = data_EX.rename(columns={'Value': 'EX Price'})
data_EX['EX LN Returns'] = round(np.log(data_EX['EX Price']).diff(),4)
data_EX.tail()

,EX Price,EX LN Returns
Date,,
2017-10-27,121.3015,0.0043
2017-10-30,121.0627,-0.0020
2017-10-31,120.9444,-0.0010
2017-11-01,120.8867,-0.0005
2017-11-02,120.6253,-0.0022


In [13]:
type(data_EX['EX Price'][2])

numpy.float64

In [14]:
#plot the returns and closing price (with different scaling) for the BTC dataset available

daily = data_EX['EX LN Returns'].cumsum().resample('D', label='right').last().ffill()
dailyframe = daily.to_frame()
closing = data_EX['EX Price']
closingframe = closing.to_frame()
closingframe.join(dailyframe).iplot(secondary_y='EX LN Returns', width=2.5)

In [15]:
#Merge BTC & EX df
merged_1 = pd.concat([BTC_df, data_EX], axis='columns')
merged_1.head()

,BTC USD Price,BTC LN Returns,EX Price,EX LN Returns
Date,,,,
2010-09-30,0.061900,NaN,99.9125,NaN
2010-10-01,0.061973,0.0012,99.4179,-0.0050
2010-10-02,0.061400,-0.0093,NaN,NaN
2010-10-03,0.061065,-0.0055,NaN,NaN
2010-10-04,0.061294,0.0038,99.5822,0.0017


In [16]:
len(merged_1)

2591

In [17]:
#Retrieve all date close prices and plot
#Eikon crashes while processing longer time-frames, therefore they have been divided 
#into four subsamples and then merged again
#the data cannot be retrieved from Quandl or together and therefore another process 
#would imply to download the series separetely and merge together per column with the same result

overall_var = ['.SPX', 'XAU=', '.VIX', 'US3MT=RR']

df_1 = ek.get_timeseries(overall_var, fields='CLOSE',
                         start_date='2010-09-30',
                         end_date='2012-11-01', interval='daily')

df_2 = ek.get_timeseries(overall_var, fields='CLOSE',
                         start_date='2012-11-02',
                         end_date='2014-11-01', interval='daily')

df_3 = ek.get_timeseries(overall_var, fields='CLOSE',
                         start_date='2014-11-02',
                         end_date='2016-11-01', interval='daily')

df_4 = ek.get_timeseries(overall_var, fields='CLOSE',
                         start_date='2016-11-01',
                         end_date='2017-11-01', interval='daily')



In [66]:
frames = [df_1, df_2, df_3, df_4]
result = pd.concat(frames)
result.tail()

CLOSE,.SPX,XAU=,.VIX,US3MT=RR
Date,,,,
2017-10-26,2560.40,1266.73,11.30,1.108
2017-10-27,2581.07,1272.60,9.80,1.100
2017-10-30,2572.83,1275.86,10.50,1.103
2017-10-31,2575.26,1271.20,10.18,1.154
2017-11-01,2579.36,1274.01,10.20,1.167


In [67]:
result['SPX LN Returns'] = round(np.log(result['.SPX']).diff(),4)
result['GOLD LN Returns'] = round(np.log(result['XAU=']).diff(),4)
result['VIX LN Returns'] = round(np.log(result['.VIX']).diff(),4)
result['3MTBILL LN Returns'] = round((result['US3MT=RR']).diff(),4)
result.head()

CLOSE,.SPX,XAU=,.VIX,US3MT=RR,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns
Date,,,,,,,,
2010-09-30,1141.20,1308.50,23.70,0.162,NaN,NaN,NaN,NaN
2010-10-01,1146.24,1317.30,22.50,0.157,0.0044,0.0067,-0.0520,-0.005
2010-10-04,1137.03,1314.25,23.53,0.155,-0.0081,-0.0023,0.0448,-0.002
2010-10-05,1160.75,1340.25,21.76,0.119,0.0206,0.0196,-0.0782,-0.036
2010-10-06,1159.97,1348.40,21.49,0.122,-0.0007,0.0061,-0.0125,0.003


In [68]:
result.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1849 entries, 2010-09-30 to 2017-11-01
Data columns (total 8 columns):
.SPX                  1787 non-null float64
XAU=                  1848 non-null float64
.VIX                  1787 non-null float64
US3MT=RR              1805 non-null float64
SPX LN Returns        1725 non-null float64
GOLD LN Returns       1846 non-null float64
VIX LN Returns        1725 non-null float64
3MTBILL LN Returns    1760 non-null float64
dtypes: float64(8)
memory usage: 210.0 KB


In [69]:
number = merged_1.join(result, lsuffix='_caller', rsuffix='_other')
number.tail()

,BTC USD Price,BTC LN Returns,EX Price,EX LN Returns,.SPX,XAU=,.VIX,US3MT=RR,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns
Date,,,,,,,,,,,,
2017-10-29,6174.728845,0.0691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-10-30,6119.256384,-0.0090,121.0627,-0.0020,2572.83,1275.86,10.50,1.103,-0.0032,0.0026,0.069,0.003
2017-10-31,6428.514637,0.0493,120.9444,-0.0010,2575.26,1271.20,10.18,1.154,0.0009,-0.0037,-0.031,0.051
2017-11-01,6726.411835,0.0453,120.8867,-0.0005,2579.36,1274.01,10.20,1.167,0.0016,0.0022,0.002,0.013
2017-11-02,7043.777939,0.0461,120.6253,-0.0022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
len(number)

2592

In [71]:
#Retrieve F-F Momentum Daily data from their website (http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_mom_factor_daily.html)
#Processed & Cleaned
MOM_file = "/Users/Paolo/Desktop/MOMFM.xlsx"
MOM_df = pd.read_excel(MOM_file)
#MOM_df.tail()
MOM_df = MOM_df.set_index('Date')
MOM_df.head()

,MOM
Date,
2010/09/29,-0.02
2010/09/30,-0.54
2010/10/01,-0.29
2010/10/04,0.40
2010/10/05,0.24


In [72]:
overalldataset = number.join(MOM_df, lsuffix='_caller', rsuffix='_other')
overalldataset.head(10)

,BTC USD Price,BTC LN Returns,EX Price,EX LN Returns,.SPX,XAU=,.VIX,US3MT=RR,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns,MOM
Date,,,,,,,,,,,,,
2010-09-30,0.061900,NaN,99.9125,NaN,1141.20,1308.50,23.70,0.162,NaN,NaN,NaN,NaN,-0.54
2010-10-01,0.061973,0.0012,99.4179,-0.0050,1146.24,1317.30,22.50,0.157,0.0044,0.0067,-0.0520,-0.005,-0.29
2010-10-02,0.061400,-0.0093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-03,0.061065,-0.0055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-04,0.061294,0.0038,99.5822,0.0017,1137.03,1314.25,23.53,0.155,-0.0081,-0.0023,0.0448,-0.002,0.40
2010-10-05,0.061400,0.0017,99.1550,-0.0043,1160.75,1340.25,21.76,0.119,0.0206,0.0196,-0.0782,-0.036,0.24
2010-10-06,0.062810,0.0227,98.7248,-0.0043,1159.97,1348.40,21.49,0.122,-0.0007,0.0061,-0.0125,0.003,-0.62
2010-10-07,0.067000,0.0646,98.8508,0.0013,1158.06,1332.70,21.56,0.122,-0.0016,-0.0117,0.0033,0.000,0.24
2010-10-08,0.084453,0.2315,98.6701,-0.0018,1165.15,1346.30,20.71,0.122,0.0061,0.0102,-0.0402,0.000,-0.04


In [73]:
len(overalldataset)

2592

## Relationship of BTC price with Google Search Engine
<a id="second"></a>
[Index](#contents)

In [26]:
#Retrieve Google sentiment for Bitcoin keywork 
#Google does not allow to retrieve daily data from its website from longer time lags, 
#nor does the pytrends library. The only way is to process sub-batches

pytrends = TrendReq(hl='en-US', tz=360)
kw_list = ["Bitcoin"]

pytrends.build_payload(kw_list, cat=0, timeframe='2010-09-28 2010-12-31', geo='', gprop='')
interest_over_time_df_2 = pytrends.interest_over_time()

pytrends.build_payload(kw_list, cat=0, timeframe='2011-01-01 2011-06-01', geo='', gprop='')
interest_over_time_df_3 = pytrends.interest_over_time()

pytrends.build_payload(kw_list, cat=0, timeframe='2011-06-02 2011-12-31', geo='', gprop='')
interest_over_time_df_4 = pytrends.interest_over_time()

pytrends.build_payload(kw_list, cat=0, timeframe='2012-01-01 2012-06-01', geo='', gprop='')
interest_over_time_df_5 = pytrends.interest_over_time()

pytrends.build_payload(kw_list, cat=0, timeframe='2012-06-02 2012-12-31', geo='', gprop='')
interest_over_time_df_6 = pytrends.interest_over_time()

pytrends.build_payload(kw_list, cat=0, timeframe='2013-01-01 2013-06-01', geo='', gprop='')
interest_over_time_df_7 = pytrends.interest_over_time()

pytrends.build_payload(kw_list, cat=0, timeframe='2013-06-02 2013-12-31', geo='', gprop='')
interest_over_time_df_8 = pytrends.interest_over_time()

pytrends.build_payload(kw_list, cat=0, timeframe='2014-01-01 2014-06-01', geo='', gprop='')
interest_over_time_df_9 = pytrends.interest_over_time()

pytrends.build_payload(kw_list, cat=0, timeframe='2014-06-02 2014-12-31', geo='', gprop='')
interest_over_time_df_10 = pytrends.interest_over_time()

pytrends.build_payload(kw_list, cat=0, timeframe='2015-01-01 2015-06-01', geo='', gprop='')
interest_over_time_df_11 = pytrends.interest_over_time()

pytrends.build_payload(kw_list, cat=0, timeframe='2015-06-02 2015-12-31', geo='', gprop='')
interest_over_time_df_12 = pytrends.interest_over_time()

pytrends.build_payload(kw_list, cat=0, timeframe='2016-01-01 2016-06-01', geo='', gprop='')
interest_over_time_df_13 = pytrends.interest_over_time()

pytrends.build_payload(kw_list, cat=0, timeframe='2016-06-02 2016-12-31', geo='', gprop='')
interest_over_time_df_14 = pytrends.interest_over_time()

pytrends.build_payload(kw_list, cat=0, timeframe='2017-01-01 2017-06-01', geo='', gprop='')
interest_over_time_df_15 = pytrends.interest_over_time()

pytrends.build_payload(kw_list, cat=0, timeframe='2017-06-01 2017-11-01', geo='', gprop='')
interest_over_time_df_16 = pytrends.interest_over_time()


In [27]:
frames_google = [interest_over_time_df_2, 
                interest_over_time_df_3, interest_over_time_df_5, interest_over_time_df_6,
                interest_over_time_df_7, interest_over_time_df_8, interest_over_time_df_9,
                interest_over_time_df_10, interest_over_time_df_11, interest_over_time_df_12,
                interest_over_time_df_13, interest_over_time_df_14, interest_over_time_df_15, 
                interest_over_time_df_16]
result_google = pd.concat(frames_google)
result_google.head()

,Bitcoin,isPartial
date,,
2010-09-28,17,False
2010-09-29,0,False
2010-09-30,17,False
2010-10-01,18,False
2010-10-02,33,False


In [28]:
len(result_google)

2380

In [29]:
#del(result_google[2])
#del(BTC_df['BTC LN Returns'])
merge_google_final = result_google.join(BTC_df, lsuffix='_caller', rsuffix='_other')
merge_google_final.tail()

,Bitcoin,isPartial,BTC USD Price,BTC LN Returns
2017-10-28,52,False,5762.388577,-0.0017
2017-10-29,56,False,6174.728845,0.0691
2017-10-30,66,False,6119.256384,-0.0090
2017-10-31,78,False,6428.514637,0.0493
2017-11-01,96,False,6726.411835,0.0453


In [30]:
daily_prices = merge_google_final['BTC USD Price']
dailyframe_btc = daily_prices.to_frame()
bitcoin_graph = merge_google_final['Bitcoin']
closingframe_goog = bitcoin_graph.to_frame()
closingframe_goog.join(dailyframe_btc).iplot(secondary_y='Bitcoin', width=2.5)

## Descriptive Statistics
<a id="aa"></a>
[Index](#contents)

In [74]:
returns_overalldataset = overalldataset[["BTC LN Returns","EX LN Returns", "SPX LN Returns", "GOLD LN Returns", "VIX LN Returns", "3MTBILL LN Returns", "MOM"]]
description_statistics = returns_overalldataset.describe()
description_statistics

,BTC LN Returns,EX LN Returns,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns,MOM
count,2591.000000,1765.000000,1725.000000,1846.000000,1725.000000,1760.000000,1788.000000
mean,0.004510,0.000107,0.000426,-0.000017,-0.001401,0.000503,0.018294
std,0.057747,0.003083,0.009022,0.010251,0.074414,0.013634,0.695542
min,-0.664900,-0.019900,-0.069000,-0.088800,-0.314100,-0.074000,-3.170000
25%,-0.011550,-0.001600,-0.003200,-0.005200,-0.041900,-0.005000,-0.350000
50%,0.002300,0.000100,0.000500,0.000200,-0.006200,0.000000,0.050000
75%,0.021950,0.001800,0.004900,0.005400,0.033900,0.005100,0.410000
max,0.436700,0.017300,0.046300,0.046900,0.405500,0.094000,3.640000


In [215]:
returns_overalldataset.corr().iplot(kind='heatmap', colorscale='blues')

In [75]:
#Test for Skewness
skewness = returns_overalldataset.skew(axis = 0, skipna = True) 
skewness

BTC LN Returns       -0.714880
EX LN Returns         0.205639
SPX LN Returns       -0.539667
GOLD LN Returns      -0.603918
VIX LN Returns        0.806457
3MTBILL LN Returns    0.268343
MOM                  -0.234358
dtype: float64

In [78]:
#Test for Kurtosis
kurtosis = returns_overalldataset.kurtosis(axis = 0, skipna = True) 
kurtosis

BTC LN Returns        17.834047
EX LN Returns          3.503695
SPX LN Returns         5.466078
GOLD LN Returns        5.666636
VIX LN Returns         4.061406
3MTBILL LN Returns     5.997973
MOM                    2.003906
dtype: float64

In [79]:
#JB numpy & stats package for Python works fine with np.array. Still, the arithmetic computation
#has proven to be more accurate with dataframe
#Not looping for clarity of output, will loop PP 
#JB FOR BTC
skewness_BTC = skewness["BTC LN Returns"]
kurtosis_BTC = kurtosis["BTC LN Returns"]
JB_BTC = returns_overalldataset["BTC LN Returns"].count()*(((skewness_BTC**2)/6)+(((kurtosis_BTC-3)**2)/24))
#JB FOR S&P500
skewness_SPX = skewness["SPX LN Returns"]
kurtosis_SPX = kurtosis["SPX LN Returns"]
JB_SPX = returns_overalldataset["SPX LN Returns"].count()*(((skewness_SPX**2)/6)+(((kurtosis_SPX-3)**2)/24))
#JB FOR EX
skewness_EX = skewness["EX LN Returns"]
kurtosis_EX = kurtosis["EX LN Returns"]
JB_EX = returns_overalldataset["EX LN Returns"].count()*(((skewness_EX**2)/6)+(((kurtosis_EX-3)**2)/24))
#JB FOR GOLD
skewness_GOLD = skewness["GOLD LN Returns"]
kurtosis_GOLD = kurtosis["GOLD LN Returns"]
JB_GOLD = returns_overalldataset["GOLD LN Returns"].count()*(((skewness_GOLD**2)/6)+(((kurtosis_GOLD-3)**2)/24))
#JB FOR VIX
skewness_VIX = skewness["VIX LN Returns"]
kurtosis_VIX = kurtosis["VIX LN Returns"]
JB_VIX = returns_overalldataset["VIX LN Returns"].count()*(((skewness_VIX**2)/6)+(((kurtosis_VIX-3)**2)/24))
#JB FOR TBILL
skewness_3MTBILL = skewness["3MTBILL LN Returns"]
kurtosis_3MTBILL = kurtosis["3MTBILL LN Returns"]
JB_3MTBILL = returns_overalldataset["3MTBILL LN Returns"].count()*(((skewness_3MTBILL**2)/6)+(((kurtosis_3MTBILL-3)**2)/24))
#JB FOR MOM
skewness_MOM = skewness["MOM"]
kurtosis_MOM = kurtosis["MOM"]
JB_MOM = returns_overalldataset["MOM"].count()*(((skewness_MOM**2)/6)+(((kurtosis_MOM-3)**2)/24))


print(f"The JB Test for BTC is: {round(JB_BTC,2)}")
print(f"The JB Test for S&P500 is: {round(JB_SPX,2)}")
print(f"The JB Test for EX is: {round(JB_EX,2)}")
print(f"The JB Test for GOLD is: {round(JB_GOLD,2)}")
print(f"The JB Test for VIX is: {round(JB_VIX,2)}")
print(f"The JB Test for 3M T-BILL X is: {round(JB_3MTBILL,2)}")
print(f"The JB Test for MOM X is: {round(JB_MOM,2)}")

The JB Test for BTC is: 23976.81
The JB Test for S&P500 is: 520.84
The JB Test for EX is: 31.1
The JB Test for GOLD is: 659.16
The JB Test for VIX is: 267.96
The JB Test for 3M T-BILL X is: 680.23
The JB Test for MOM X is: 90.29


## Unit Root Tests: ADF, PP & KPSS
<a id="fourth"></a>
[Index](#contents)

Unit root test for non-stationarity of variables 
- Null Hypothesis: The series has a unit root (value of a =1)
- Alternate Hypothesis: The series has no unit root.
If we fail to reject the null hypothesis, we can say that the series is non-stationary, i.e. if p-value <= 0.05 we do reject the null hypothesis (H0) and the data does not have a unit root (it is stationary).


In [80]:
#Not looping now for clarification purposes (i.e. associate each ADF test to each column)
from statsmodels.tsa.stattools import adfuller
series = returns_overalldataset["BTC LN Returns"].dropna()
X = series.values
result = adfuller(X)
print('ADF Statistic for BTC: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))
    
series = returns_overalldataset["GOLD LN Returns"].dropna()
X = series.values
result = adfuller(X)
print('ADF Statistic for GOLD: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))
    
series = returns_overalldataset["EX LN Returns"].dropna()
X = series.values
result = adfuller(X)
print('ADF Statistic for EX: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))
    
series = returns_overalldataset["SPX LN Returns"].dropna()
X = series.values
result = adfuller(X)
print('ADF Statistic SPX: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))
    
    
series = returns_overalldataset["VIX LN Returns"].dropna()
X = series.values
result = adfuller(X)
print('ADF Statistic VIX: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))
    
series = returns_overalldataset["MOM"].dropna()
X = series.values
result = adfuller(X)
print('ADF Statistic MOM: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))
    
series = returns_overalldataset["3MTBILL LN Returns"].dropna()
X = series.values
result = adfuller(X)
print('ADF Statistic 3MTBILL: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic for BTC: -11.290039
p-value: 0.000000
Critical Values:
	1%: -3.433
	5%: -2.863
	10%: -2.567
ADF Statistic for GOLD: -43.148808
p-value: 0.000000
Critical Values:
	1%: -3.434
	5%: -2.863
	10%: -2.568
ADF Statistic for EX: -14.279236
p-value: 0.000000
Critical Values:
	1%: -3.434
	5%: -2.863
	10%: -2.568
ADF Statistic SPX: -21.772709
p-value: 0.000000
Critical Values:
	1%: -3.434
	5%: -2.863
	10%: -2.568
ADF Statistic VIX: -14.525600
p-value: 0.000000
Critical Values:
	1%: -3.434
	5%: -2.863
	10%: -2.568
ADF Statistic MOM: -25.288728
p-value: 0.000000
Critical Values:
	1%: -3.434
	5%: -2.863
	10%: -2.568
ADF Statistic 3MTBILL: -12.632650
p-value: 0.000000
Critical Values:
	1%: -3.434
	5%: -2.863
	10%: -2.568


Phillips-Perron test makes correction to the test statistics and is robust to the unspecified autocorrelation and heteroscedasticity in the error. We have considered both PP and KPSS tests. Note that for ADF, the 2nd value returned is the p-value associated with the null that there *is* a unit root. For KPSS, the 2nd value returns is the p-value associated with the null of stationarity. 

In [81]:
#PP Test
#Same order of columns as for ADF test 

for columns in returns_overalldataset:
    try:
        print(PhillipsPerron(returns_overalldataset[columns].dropna()))
    except:
        print("Validated")

     Phillips-Perron Test (Z-tau)    
Test Statistic                -49.339
P-value                         0.000
Lags                               28
-------------------------------------

Trend: Constant
Critical Values: -3.43 (1%), -2.86 (5%), -2.57 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary.
     Phillips-Perron Test (Z-tau)    
Test Statistic                -40.606
P-value                         0.000
Lags                               25
-------------------------------------

Trend: Constant
Critical Values: -3.43 (1%), -2.86 (5%), -2.57 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary.
     Phillips-Perron Test (Z-tau)    
Test Statistic                -45.098
P-value                         0.000
Lags                               25
-------------------------------------

Trend: Constant
Critical Values: -3.43 (1%), -2.86 (5%), -2.57 (10%)
Nu

In [82]:
#Recap of PP tests for clarity purposes
columns_list1 = list(returns_overalldataset)
for columns in returns_overalldataset:
    i = PhillipsPerron(returns_overalldataset[columns].dropna())
    try:
        print("The PP test is {}".format(i.stat))
    except:
        print("Validated")

The PP test is -49.338968691523576
The PP test is -40.606461380501436
The PP test is -45.09802269091106
The PP test is -43.17241122036643
The PP test is -48.92716413206144
The PP test is -43.37938269747539
The PP test is -39.30341063824439


In [83]:
#KPSS Test: same order of columns as for ADF test 
for columns in returns_overalldataset:
    print(KPSS(returns_overalldataset[columns].dropna()))

    KPSS Stationarity Test Results   
Test Statistic                  0.443
P-value                         0.056
Lags                               10
-------------------------------------

Trend: Constant
Critical Values: 0.74 (1%), 0.46 (5%), 0.35 (10%)
Null Hypothesis: The process is weakly stationary.
Alternative Hypothesis: The process contains a unit root.
    KPSS Stationarity Test Results   
Test Statistic                  0.158
P-value                         0.366
Lags                               13
-------------------------------------

Trend: Constant
Critical Values: 0.74 (1%), 0.46 (5%), 0.35 (10%)
Null Hypothesis: The process is weakly stationary.
Alternative Hypothesis: The process contains a unit root.
    KPSS Stationarity Test Results   
Test Statistic                  0.043
P-value                         0.918
Lags                               27
-------------------------------------

Trend: Constant
Critical Values: 0.74 (1%), 0.46 (5%), 0.35 (10%)
Null Hypoth

Stationarity Test Comment:
The results for Dickey-Fuller and Phillips-Perron unit root tests confirm that our dataset is stationary in all tests performed. 

## Dummy Variable Introduction 
<a id="fifth"></a>
[Index](#contents)

In [ ]:
#Day of the week variables

In [188]:
#Introduce day-of-the-week computation 
#Retrieve BTC data from Coin Metric (https://coinmetrics.io/community-data-dictionary/)
#Processed & Cleaned
BTC_file = "/Users/Paolo/Desktop/BTC.xlsx"
BTC_df = pd.read_excel(BTC_file)
BTC_df

,Date,BTC USD Price
0,2010-09-30,0.061900
1,2010-10-01,0.061973
2,2010-10-02,0.061400
3,2010-10-03,0.061065
4,2010-10-04,0.061294
...,...,...
2586,2017-10-29,6174.728845
2587,2017-10-30,6119.256384
2588,2017-10-31,6428.514637
2589,2017-11-01,6726.411835


In [189]:
#the week starts on Monday, which is denoted by 0 and ends on Sunday which is denoted by 6.
result = BTC_df["Date"].dt.dayofweek 
column_day = result.to_frame()
column_day 

,Date
0,3
1,4
2,5
3,6
4,0
...,...
2586,6
2587,0
2588,1
2589,2


In [190]:
finaldb_BTC = BTC_df.join(column_day, lsuffix='_caller', rsuffix='_other')
finaldb_BTC = finaldb_BTC.set_index('Date_caller')
del(finaldb_BTC['BTC USD Price'])

In [191]:
finaldb_BTC

,Date_other
Date_caller,
2010-09-30,3
2010-10-01,4
2010-10-02,5
2010-10-03,6
2010-10-04,0
...,...
2017-10-29,6
2017-10-30,0
2017-10-31,1


In [283]:
finaldb = returns_overalldataset.join(finaldb_BTC, lsuffix='_caller', rsuffix='_other')
finaldb = finaldb.rename(columns={'Date_other': 'Day-of-the-week'})
finaldb

,BTC LN Returns,EX LN Returns,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns,MOM,Day-of-the-week
2010-09-30,NaN,NaN,NaN,NaN,NaN,NaN,-0.54,3
2010-10-01,0.0012,-0.0050,0.0044,0.0067,-0.0520,-0.005,-0.29,4
2010-10-02,-0.0093,NaN,NaN,NaN,NaN,NaN,NaN,5
2010-10-03,-0.0055,NaN,NaN,NaN,NaN,NaN,NaN,6
2010-10-04,0.0038,0.0017,-0.0081,-0.0023,0.0448,-0.002,0.40,0
...,...,...,...,...,...,...,...,...
2017-10-29,0.0691,NaN,NaN,NaN,NaN,NaN,NaN,6
2017-10-30,-0.0090,-0.0020,-0.0032,0.0026,0.0690,0.003,-0.17,0
2017-10-31,0.0493,-0.0010,0.0009,-0.0037,-0.0310,0.051,0.42,1
2017-11-01,0.0453,-0.0005,0.0016,0.0022,0.0020,0.013,-0.54,2


In [284]:
#Creation of dummy variables
dummy = pd.get_dummies(finaldb['Day-of-the-week'])
#print(dummy.columns.tolist())
dummy = dummy.rename(columns={0: 'M'})
dummy = dummy.rename(columns={1: 'T'})
dummy = dummy.rename(columns={2: 'W'})
dummy = dummy.rename(columns={3: 'TH'})
del(dummy[4])
del(dummy[5])
del(dummy[6])
dummy.tail()

,M,T,W,TH
2017-10-29,0,0,0,0
2017-10-30,1,0,0,0
2017-10-31,0,1,0,0
2017-11-01,0,0,1,0
2017-11-02,0,0,0,1


In [285]:
finaldb_dummy = finaldb.join(dummy, lsuffix='_caller', rsuffix='_other')
finaldb_dummy

,BTC LN Returns,EX LN Returns,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns,MOM,Day-of-the-week,M,T,W,TH
2010-09-30,NaN,NaN,NaN,NaN,NaN,NaN,-0.54,3,0,0,0,1
2010-10-01,0.0012,-0.0050,0.0044,0.0067,-0.0520,-0.005,-0.29,4,0,0,0,0
2010-10-02,-0.0093,NaN,NaN,NaN,NaN,NaN,NaN,5,0,0,0,0
2010-10-03,-0.0055,NaN,NaN,NaN,NaN,NaN,NaN,6,0,0,0,0
2010-10-04,0.0038,0.0017,-0.0081,-0.0023,0.0448,-0.002,0.40,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2017-10-29,0.0691,NaN,NaN,NaN,NaN,NaN,NaN,6,0,0,0,0
2017-10-30,-0.0090,-0.0020,-0.0032,0.0026,0.0690,0.003,-0.17,0,1,0,0,0
2017-10-31,0.0493,-0.0010,0.0009,-0.0037,-0.0310,0.051,0.42,1,0,1,0,0
2017-11-01,0.0453,-0.0005,0.0016,0.0022,0.0020,0.013,-0.54,2,0,0,1,0


In [286]:
finaldb_dummy["MOM"]

2010-09-30   -0.54
2010-10-01   -0.29
2010-10-02     NaN
2010-10-03     NaN
2010-10-04    0.40
              ... 
2017-10-29     NaN
2017-10-30   -0.17
2017-10-31    0.42
2017-11-01   -0.54
2017-11-02    0.85
Name: MOM, Length: 2594, dtype: float64

In [287]:
finaldb_dummy["BTC LN Returns -1"] = finaldb_dummy["BTC LN Returns"].shift(1)

In [288]:
finaldb_dummy

,BTC LN Returns,EX LN Returns,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns,MOM,Day-of-the-week,M,T,W,TH,BTC LN Returns -1
2010-09-30,NaN,NaN,NaN,NaN,NaN,NaN,-0.54,3,0,0,0,1,NaN
2010-10-01,0.0012,-0.0050,0.0044,0.0067,-0.0520,-0.005,-0.29,4,0,0,0,0,NaN
2010-10-02,-0.0093,NaN,NaN,NaN,NaN,NaN,NaN,5,0,0,0,0,0.0012
2010-10-03,-0.0055,NaN,NaN,NaN,NaN,NaN,NaN,6,0,0,0,0,-0.0093
2010-10-04,0.0038,0.0017,-0.0081,-0.0023,0.0448,-0.002,0.40,0,1,0,0,0,-0.0055
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-10-29,0.0691,NaN,NaN,NaN,NaN,NaN,NaN,6,0,0,0,0,-0.0017
2017-10-30,-0.0090,-0.0020,-0.0032,0.0026,0.0690,0.003,-0.17,0,1,0,0,0,0.0691
2017-10-31,0.0493,-0.0010,0.0009,-0.0037,-0.0310,0.051,0.42,1,0,1,0,0,-0.0090
2017-11-01,0.0453,-0.0005,0.0016,0.0022,0.0020,0.013,-0.54,2,0,0,1,0,0.0493


## OLS Regressions
<a id="six"></a>
[Index](#contents)
### Period 1: Panel A
<a id="7"></a>
First sub-sample to run OLS (October 2010-December 2012) 

In [289]:
#Period 1: PANEL A
#truncate dataset to create first sub-sample 
finaldb_dummy_period1 = finaldb_dummy.truncate(after=pd.Timestamp('2012-12-31'))
finaldb_dummy_period1.tail()

,BTC LN Returns,EX LN Returns,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns,MOM,Day-of-the-week,M,T,W,TH,BTC LN Returns -1
2012-12-27,-0.0041,0.0004,-0.0012,0.0021,-0.0005,-0.0101,0.28,3,0,0,0,1,0.0066
2012-12-28,0.0042,-0.0007,-0.0111,-0.0051,0.1544,-0.0660,0.33,4,0,0,0,0,-0.0041
2012-12-29,-0.0061,NaN,NaN,NaN,NaN,NaN,NaN,5,0,0,0,0,0.0042
2012-12-30,0.0053,NaN,NaN,NaN,NaN,NaN,NaN,6,0,0,0,0,-0.0061
2012-12-31,0.0074,0.0005,0.0168,0.0117,-0.2318,0.0355,0.24,0,1,0,0,0,0.0053


In [299]:
#Model 1
finaldb_dummy_period1 = finaldb_dummy_period1.dropna()
finaldb_dummy_period1["intercept"] = 1
results_OLS_1 = sm.OLS(finaldb_dummy_period1["BTC LN Returns"], finaldb_dummy_period1[["intercept", "M", "T", "W", "TH"]]).fit()

print(results_OLS_1.summary())

#The results are not significant

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.370
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.243
Time:                        12:55:18   Log-Likelihood:                 646.74
No. Observations:                 536   AIC:                            -1283.
Df Residuals:                     531   BIC:                            -1262.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0058      0.007      0.832      0.4

In [300]:
#Model 2
#The only significant result is the one given by BTC LN Returns -1 
#(as the p-value is lower than the threshold)
finaldb_dummy = finaldb_dummy.dropna()
finaldb_dummy["intercept"] = 1
results_OLS_2 = sm.OLS(finaldb_dummy["BTC LN Returns"], 
                     finaldb_dummy[["intercept", "M", "T", "W", "TH", "BTC LN Returns -1"]]).fit()

print(results_OLS_2.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.866
Date:                Sun, 24 Nov 2019   Prob (F-statistic):             0.0972
Time:                        12:55:27   Log-Likelihood:                 2324.5
No. Observations:                1676   AIC:                            -4637.
Df Residuals:                    1670   BIC:                            -4604.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0031      0.00

In [301]:
#Model 3
#The only significant result is the one given by BTC LN Returns -1 
#(as the p-value is lower than the threshold)
finaldb_dummy = finaldb_dummy.dropna()
finaldb_dummy["intercept"] = 1
results_OLS_3 = sm.OLS(finaldb_dummy["BTC LN Returns"], 
                     finaldb_dummy[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", "SPX LN Returns"]]).fit()

print(results_OLS_3.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.828
Date:                Sun, 24 Nov 2019   Prob (F-statistic):             0.0901
Time:                        12:55:33   Log-Likelihood:                 2325.3
No. Observations:                1676   AIC:                            -4637.
Df Residuals:                    1669   BIC:                            -4599.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0030      0.00

In [302]:
#Model 4
#The only significant result is the one given by BTC LN Returns -1 
#(as the p-value is lower than the threshold)
finaldb_dummy = finaldb_dummy.dropna()
finaldb_dummy["intercept"] = 1
results_OLS_4 = sm.OLS(finaldb_dummy["BTC LN Returns"], 
                     finaldb_dummy[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns", 
                                    "GOLD LN Returns"]]).fit()

print(results_OLS_4.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.568
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.141
Time:                        12:55:39   Log-Likelihood:                 2325.3
No. Observations:                1676   AIC:                            -4635.
Df Residuals:                    1668   BIC:                            -4591.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0030      0.00

In [303]:
#Model 5
#The only significant result is the one given by BTC LN Returns -1 
#(as the p-value is lower than the threshold)
finaldb_dummy = finaldb_dummy.dropna()
finaldb_dummy["intercept"] = 1
results_OLS_5 = sm.OLS(finaldb_dummy["BTC LN Returns"], 
                     finaldb_dummy[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns", 
                                    "GOLD LN Returns", 
                                    "MOM"]]).fit()

print(results_OLS_5.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.456
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.169
Time:                        12:55:43   Log-Likelihood:                 2325.7
No. Observations:                1676   AIC:                            -4633.
Df Residuals:                    1667   BIC:                            -4585.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0031      0.00

In [304]:
#Model 6
#The only significant result is the one given by BTC LN Returns -1 
#(as the p-value is lower than the threshold)
finaldb_dummy = finaldb_dummy.dropna()
finaldb_dummy["intercept"] = 1
results_OLS_6 = sm.OLS(finaldb_dummy["BTC LN Returns"], 
                     finaldb_dummy[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns", 
                                    "GOLD LN Returns", 
                                    "MOM", 
                                    "3MTBILL LN Returns"]]).fit()

print(results_OLS_6.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.300
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.232
Time:                        12:55:49   Log-Likelihood:                 2325.7
No. Observations:                1676   AIC:                            -4631.
Df Residuals:                    1666   BIC:                            -4577.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
intercept              0.0031      0

In [305]:
#Model 7
#The only significant result is the one given by BTC LN Returns -1 
#(as the p-value is lower than the threshold)
finaldb_dummy = finaldb_dummy.dropna()
finaldb_dummy["intercept"] = 1
results_OLS_7 = sm.OLS(finaldb_dummy["BTC LN Returns"], 
                     finaldb_dummy[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns", 
                                    "GOLD LN Returns", 
                                    "MOM", 
                                    "3MTBILL LN Returns", 
                                    "EX LN Returns"]]).fit()

print(results_OLS_7.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.242
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.259
Time:                        12:55:55   Log-Likelihood:                 2326.1
No. Observations:                1676   AIC:                            -4630.
Df Residuals:                    1665   BIC:                            -4570.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
intercept              0.0030      0

In [306]:
#Model 8
#The only significant result is the one given by BTC LN Returns -1 
#(as the p-value is lower than the threshold)
finaldb_dummy = finaldb_dummy.dropna()
finaldb_dummy["intercept"] = 1
results_OLS_8 = sm.OLS(finaldb_dummy["BTC LN Returns"], 
                     finaldb_dummy[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns", 
                                    "GOLD LN Returns", 
                                    "MOM", 
                                    "3MTBILL LN Returns", 
                                    "EX LN Returns", 
                                    "VIX LN Returns"]]).fit()

print(results_OLS_8.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.131
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.332
Time:                        12:56:01   Log-Likelihood:                 2326.1
No. Observations:                1676   AIC:                            -4628.
Df Residuals:                    1664   BIC:                            -4563.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
intercept              0.0030      0

The findings of the original paper were significant for two values, with respect to the S&P500 returns in the Model (3) and Model (4). This show that a contemporaneous change in the S&P500 Index has a positive relationship with Bitcoin's price. Nonetheless, in our model the results are not significant, therefore we cannot state the same conclusion. 

Moreover, when the authors added the VIX to Model (8), they observed the significant negative relationship with it, which may imply that Bitcoin can be a hedge when VIX (uncertainty) increases. Interestingly, they did not find a similar relationship with respect to a change in the price of gold, which is familiar as a hedge against uncertainty in the context of equities. 
We did find a negative relationship with GOLD in Model (4, 5, 6), nevertheless, we do not have any significance apart for the lag-value of BTC returns (which is consistent with the random walk hypothesis). 

### Period 1: Panel B
<a id="8"></a>
[Index](#contents)

In [325]:
finaldb_dummy["BTC LN Returns -1"] = finaldb_dummy["BTC LN Returns"].shift(1)
finaldb_dummy["SPX LN Returns -1"] = finaldb_dummy["SPX LN Returns"].shift(1)
finaldb_dummy["GOLD LN Returns -1"] = finaldb_dummy["GOLD LN Returns"].shift(1)
finaldb_dummy["MOM -1"] = finaldb_dummy["MOM"].shift(1)
finaldb_dummy["3MTBILL LN Returns -1"] = finaldb_dummy["3MTBILL LN Returns"].shift(1)
finaldb_dummy["EX LN Returns -1"] = finaldb_dummy["EX LN Returns"].shift(1)
finaldb_dummy["VIX LN Returns -1"] = finaldb_dummy["VIX LN Returns"].shift(1)
finaldb_dummy

,BTC LN Returns,EX LN Returns,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns,MOM,Day-of-the-week,M,T,W,TH,BTC LN Returns -1,intercept,SPX LN Returns -1,GOLD LN Returns -1,MOM -1,3MTBILL LN Returns -1,EX LN Returns -1,VIX LN Returns -1
2010-10-04,0.0038,0.0017,-0.0081,-0.0023,0.0448,-0.002,0.40,0,1,0,0,0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-05,0.0017,-0.0043,0.0206,0.0196,-0.0782,-0.036,0.24,1,0,1,0,0,0.0038,1,-0.0081,-0.0023,0.40,-0.002,0.0017,0.0448
2010-10-06,0.0227,-0.0043,-0.0007,0.0061,-0.0125,0.003,-0.62,2,0,0,1,0,0.0017,1,0.0206,0.0196,0.24,-0.036,-0.0043,-0.0782
2010-10-07,0.0646,0.0013,-0.0016,-0.0117,0.0033,0.000,0.24,3,0,0,0,1,0.0227,1,-0.0007,0.0061,-0.62,0.003,-0.0043,-0.0125
2010-10-08,0.2315,-0.0018,0.0061,0.0102,-0.0402,0.000,-0.04,4,0,0,0,0,0.0646,1,-0.0016,-0.0117,0.24,0.000,0.0013,0.0033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-10-26,0.0282,0.0020,0.0013,-0.0081,0.0062,-0.003,0.34,3,0,0,0,1,0.0388,1,-0.0047,0.0005,-0.10,-0.015,-0.0001,0.0063
2017-10-27,-0.0220,0.0043,0.0080,0.0046,-0.1424,-0.008,1.02,4,0,0,0,0,0.0282,1,0.0013,-0.0081,0.34,-0.003,0.0020,0.0062
2017-10-30,-0.0090,-0.0020,-0.0032,0.0026,0.0690,0.003,-0.17,0,1,0,0,0,-0.0220,1,0.0080,0.0046,1.02,-0.008,0.0043,-0.1424
2017-10-31,0.0493,-0.0010,0.0009,-0.0037,-0.0310,0.051,0.42,1,0,1,0,0,-0.0090,1,-0.0032,0.0026,-0.17,0.003,-0.0020,0.0690


In [326]:
#Period 1: PANEL B 
#truncate dataset to create first sub-sample 
finaldb_dummy_period1_B = finaldb_dummy.truncate(after=pd.Timestamp('2012-12-31'))
finaldb_dummy_period1_B.tail()

,BTC LN Returns,EX LN Returns,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns,MOM,Day-of-the-week,M,T,W,TH,BTC LN Returns -1,intercept,SPX LN Returns -1,GOLD LN Returns -1,MOM -1,3MTBILL LN Returns -1,EX LN Returns -1,VIX LN Returns -1
2012-12-21,-0.0104,0.0035,-0.0094,0.0057,0.0096,0.0025,-0.25,4,0,0,0,0,0.0023,1,0.0055,-0.0117,0.21,0.0000,0.0011,0.0177
2012-12-24,0.0081,0.0009,-0.0024,0.0002,0.0412,-0.0025,0.40,0,1,0,0,0,-0.0104,1,-0.0094,0.0057,-0.25,0.0025,0.0035,0.0096
2012-12-27,-0.0041,0.0004,-0.0012,0.0021,-0.0005,-0.0101,0.28,3,0,0,0,1,0.0081,1,-0.0024,0.0002,0.40,-0.0025,0.0009,0.0412
2012-12-28,0.0042,-0.0007,-0.0111,-0.0051,0.1544,-0.0660,0.33,4,0,0,0,0,-0.0041,1,-0.0012,0.0021,0.28,-0.0101,0.0004,-0.0005
2012-12-31,0.0074,0.0005,0.0168,0.0117,-0.2318,0.0355,0.24,0,1,0,0,0,0.0042,1,-0.0111,-0.0051,0.33,-0.0660,-0.0007,0.1544


In [330]:
#Model 1
finaldb_dummy_period1_B = finaldb_dummy_period1_B.dropna()
finaldb_dummy_period1_B["intercept"] = 1
results_OLS_1_B = sm.OLS(finaldb_dummy_period1_B["BTC LN Returns"], finaldb_dummy_period1_B[["intercept", "M", "T", "W", "TH"]]).fit()

print(results_OLS_1_B.summary())

#The results are not significant

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.367
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.244
Time:                        13:52:35   Log-Likelihood:                 645.04
No. Observations:                 535   AIC:                            -1280.
Df Residuals:                     530   BIC:                            -1259.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0058      0.007      0.831      0.4

In [334]:
#Model 2
#The results are not significant with the exception of the lag-value of BTC ln returns 
finaldb_dummy_period1_B = finaldb_dummy_period1_B.dropna()
finaldb_dummy_period1_B["intercept"] = 1
results_OLS_2_B = sm.OLS(finaldb_dummy_period1_B["BTC LN Returns"], 
                     finaldb_dummy_period1_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1"]]).fit()

print(results_OLS_2_B.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     2.878
Date:                Sun, 24 Nov 2019   Prob (F-statistic):             0.0142
Time:                        13:53:03   Log-Likelihood:                 649.47
No. Observations:                 535   AIC:                            -1287.
Df Residuals:                     529   BIC:                            -1261.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0044      0.00

In [336]:
#Model 3
finaldb_dummy_period1_B = finaldb_dummy_period1_B.dropna()
finaldb_dummy_period1_B["intercept"] = 1
results_OLS_3_B = sm.OLS(finaldb_dummy_period1_B["BTC LN Returns"], 
                     finaldb_dummy_period1_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1"]]).fit()

print(results_OLS_3_B.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     2.563
Date:                Sun, 24 Nov 2019   Prob (F-statistic):             0.0187
Time:                        13:53:41   Log-Likelihood:                 649.97
No. Observations:                 535   AIC:                            -1286.
Df Residuals:                     528   BIC:                            -1256.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0048      0.00

In [391]:
#Model 4
finaldb_dummy_period1_B = finaldb_dummy_period1_B.dropna()
finaldb_dummy_period1_B["intercept"] = 1
results_OLS_4_B = sm.OLS(finaldb_dummy_period1_B["BTC LN Returns"], 
                     finaldb_dummy_period1_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns -1"]]).fit()

print(results_OLS_4_B.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     2.617
Date:                Sun, 24 Nov 2019   Prob (F-statistic):             0.0115
Time:                        14:42:27   Log-Likelihood:                 651.43
No. Observations:                 535   AIC:                            -1287.
Df Residuals:                     527   BIC:                            -1253.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
intercept              0.0049      0

In [392]:
#Model 5
finaldb_dummy_period1_B = finaldb_dummy_period1_B.dropna()
finaldb_dummy_period1_B["intercept"] = 1
results_OLS_5_B = sm.OLS(finaldb_dummy_period1_B["BTC LN Returns"], 
                     finaldb_dummy_period1_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns -1", 
                                    "MOM -1"]]).fit()

print(results_OLS_5_B.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     2.318
Date:                Sun, 24 Nov 2019   Prob (F-statistic):             0.0189
Time:                        14:42:36   Log-Likelihood:                 651.56
No. Observations:                 535   AIC:                            -1285.
Df Residuals:                     526   BIC:                            -1247.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
intercept              0.0051      0

In [393]:
#Model 6
finaldb_dummy_period1_B = finaldb_dummy_period1_B.dropna()
finaldb_dummy_period1_B["intercept"] = 1
results_OLS_6_B = sm.OLS(finaldb_dummy_period1_B["BTC LN Returns"], 
                     finaldb_dummy_period1_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns -1", 
                                    "MOM -1", 
                                    "3MTBILL LN Returns -1"]]).fit()

print(results_OLS_6_B.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     2.113
Date:                Sun, 24 Nov 2019   Prob (F-statistic):             0.0269
Time:                        14:42:44   Log-Likelihood:                 651.81
No. Observations:                 535   AIC:                            -1284.
Df Residuals:                     525   BIC:                            -1241.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
intercept                 0.00

In [394]:
#Model 7
finaldb_dummy_period1_B = finaldb_dummy_period1_B.dropna()
finaldb_dummy_period1_B["intercept"] = 1
results_OLS_7_B = sm.OLS(finaldb_dummy_period1_B["BTC LN Returns"], 
                     finaldb_dummy_period1_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns", 
                                    "MOM -1", 
                                    "3MTBILL LN Returns -1", 
                                    "EX LN Returns -1"]]).fit()

print(results_OLS_7_B.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.703
Date:                Sun, 24 Nov 2019   Prob (F-statistic):             0.0770
Time:                        14:42:53   Log-Likelihood:                 650.84
No. Observations:                 535   AIC:                            -1280.
Df Residuals:                     524   BIC:                            -1233.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
intercept                 0.00

In [390]:
#Model 8
finaldb_dummy_period1_B = finaldb_dummy_period1_B.dropna()
finaldb_dummy_period1_B["intercept"] = 1
results_OLS_8_B = sm.OLS(finaldb_dummy_period1_B["BTC LN Returns"], 
                     finaldb_dummy_period1_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns -1", 
                                    "MOM -1", 
                                    "3MTBILL LN Returns -1", 
                                    "EX LN Returns -1", 
                                    "VIX LN Returns -1"]]).fit()

print(results_OLS_8_B.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     1.758
Date:                Sun, 24 Nov 2019   Prob (F-statistic):             0.0584
Time:                        14:41:57   Log-Likelihood:                 652.00
No. Observations:                 535   AIC:                            -1280.
Df Residuals:                     523   BIC:                            -1229.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
intercept                 0.00

### Period 2: Panel A
<a id="9"></a>
[Index](#contents)
- Second sub-sample to run OLS (January 2013-May 2015) 

In [342]:
#Period 2: PANEL A
#truncate dataset to create first sub-sample 
finaldb_dummy

,BTC LN Returns,EX LN Returns,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns,MOM,Day-of-the-week,M,T,W,TH,BTC LN Returns -1,intercept,SPX LN Returns -1,GOLD LN Returns -1,MOM -1,3MTBILL LN Returns -1,EX LN Returns -1,VIX LN Returns -1
2010-10-04,0.0038,0.0017,-0.0081,-0.0023,0.0448,-0.002,0.40,0,1,0,0,0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-05,0.0017,-0.0043,0.0206,0.0196,-0.0782,-0.036,0.24,1,0,1,0,0,0.0038,1,-0.0081,-0.0023,0.40,-0.002,0.0017,0.0448
2010-10-06,0.0227,-0.0043,-0.0007,0.0061,-0.0125,0.003,-0.62,2,0,0,1,0,0.0017,1,0.0206,0.0196,0.24,-0.036,-0.0043,-0.0782
2010-10-07,0.0646,0.0013,-0.0016,-0.0117,0.0033,0.000,0.24,3,0,0,0,1,0.0227,1,-0.0007,0.0061,-0.62,0.003,-0.0043,-0.0125
2010-10-08,0.2315,-0.0018,0.0061,0.0102,-0.0402,0.000,-0.04,4,0,0,0,0,0.0646,1,-0.0016,-0.0117,0.24,0.000,0.0013,0.0033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-10-26,0.0282,0.0020,0.0013,-0.0081,0.0062,-0.003,0.34,3,0,0,0,1,0.0388,1,-0.0047,0.0005,-0.10,-0.015,-0.0001,0.0063
2017-10-27,-0.0220,0.0043,0.0080,0.0046,-0.1424,-0.008,1.02,4,0,0,0,0,0.0282,1,0.0013,-0.0081,0.34,-0.003,0.0020,0.0062
2017-10-30,-0.0090,-0.0020,-0.0032,0.0026,0.0690,0.003,-0.17,0,1,0,0,0,-0.0220,1,0.0080,0.0046,1.02,-0.008,0.0043,-0.1424
2017-10-31,0.0493,-0.0010,0.0009,-0.0037,-0.0310,0.051,0.42,1,0,1,0,0,-0.0090,1,-0.0032,0.0026,-0.17,0.003,-0.0020,0.0690


In [344]:
finaldb_dummy_period2 = finaldb_dummy.truncate(before=pd.Timestamp('2013-01-01') , after=pd.Timestamp('2015-05-31'))
finaldb_dummy_period2.head()

,BTC LN Returns,EX LN Returns,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns,MOM,Day-of-the-week,M,T,W,TH,BTC LN Returns -1,intercept,SPX LN Returns -1,GOLD LN Returns -1,MOM -1,3MTBILL LN Returns -1,EX LN Returns -1,VIX LN Returns -1
2013-01-03,0.0078,0.0011,-0.0021,-0.0140,-0.0082,-0.0051,-0.45,3,0,0,0,1,0.0074,1,0.0168,0.0117,0.24,0.0355,0.0005,-0.2318
2013-01-04,0.0050,0.0031,0.0049,-0.0040,-0.0514,0.0000,-0.36,4,0,0,0,0,0.0078,1,-0.0021,-0.0140,-0.45,-0.0051,0.0011,-0.0082
2013-01-07,0.0038,-0.0012,-0.0031,-0.0058,-0.0029,-0.0026,0.15,0,1,0,0,0,0.0050,1,0.0049,-0.0040,-0.36,0.0000,0.0031,-0.0514
2013-01-08,0.0164,0.0007,-0.0032,0.0072,-0.0124,-0.0025,0.86,1,0,1,0,0,0.0038,1,-0.0031,-0.0058,0.15,-0.0026,-0.0012,-0.0029
2013-01-09,0.0029,-0.0005,0.0027,-0.0008,0.0139,-0.0051,0.08,2,0,0,1,0,0.0164,1,-0.0032,0.0072,0.86,-0.0025,0.0007,-0.0124


In [345]:
#Period 2
#Model 1
finaldb_dummy_period2 = finaldb_dummy_period2.dropna()
finaldb_dummy_period2["intercept"] = 1
results_OLS_2_A = sm.OLS(finaldb_dummy_period2["BTC LN Returns"], finaldb_dummy_period2[["intercept", "M", "T", "W", "TH"]]).fit()

print(results_OLS_2_A.summary())

#The results are not significant

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9955
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.409
Time:                        14:04:11   Log-Likelihood:                 726.25
No. Observations:                 563   AIC:                            -1443.
Df Residuals:                     558   BIC:                            -1421.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0019      0.006      0.311      0.7

In [346]:
#Model 2
finaldb_dummy_period2 = finaldb_dummy_period2.dropna()
finaldb_dummy_period2["intercept"] = 1
results_OLS_2_A2 = sm.OLS(finaldb_dummy_period2["BTC LN Returns"], 
                     finaldb_dummy_period2[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1"]]).fit()

print(results_OLS_2_A2.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.7950
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.554
Time:                        14:07:41   Log-Likelihood:                 726.25
No. Observations:                 563   AIC:                            -1441.
Df Residuals:                     557   BIC:                            -1415.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0019      0.00

In [389]:
#Model 3
finaldb_dummy_period2 = finaldb_dummy_period2.dropna()
finaldb_dummy_period2["intercept"] = 1
results_OLS_2_A3 = sm.OLS(finaldb_dummy_period2["BTC LN Returns"], 
                     finaldb_dummy_period2[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns"]]).fit()

print(results_OLS_2_A3.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9317
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.472
Time:                        14:41:39   Log-Likelihood:                 727.07
No. Observations:                 563   AIC:                            -1440.
Df Residuals:                     556   BIC:                            -1410.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0026      0.00

In [388]:
#Model 4
finaldb_dummy_period2 = finaldb_dummy_period2.dropna()
finaldb_dummy_period2["intercept"] = 1
results_OLS_2_A4 = sm.OLS(finaldb_dummy_period2["BTC LN Returns"], 
                     finaldb_dummy_period2[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns", 
                                    "GOLD LN Returns"]]).fit()

print(results_OLS_2_A4.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9811
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.444
Time:                        14:41:35   Log-Likelihood:                 727.71
No. Observations:                 563   AIC:                            -1439.
Df Residuals:                     555   BIC:                            -1405.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0026      0.00

In [387]:
#Model 5
finaldb_dummy_period2 = finaldb_dummy_period2.dropna()
finaldb_dummy_period2["intercept"] = 1
results_OLS_2_A5 = sm.OLS(finaldb_dummy_period2["BTC LN Returns"], 
                     finaldb_dummy_period2[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns", 
                                    "GOLD LN Returns", 
                                    "MOM"]]).fit()

print(results_OLS_2_A5.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.8716
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.540
Time:                        14:41:31   Log-Likelihood:                 727.77
No. Observations:                 563   AIC:                            -1438.
Df Residuals:                     554   BIC:                            -1399.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0026      0.00

In [386]:
#Model 6
finaldb_dummy_period2 = finaldb_dummy_period2.dropna()
finaldb_dummy_period2["intercept"] = 1
results_OLS_2_A6 = sm.OLS(finaldb_dummy_period2["BTC LN Returns"], 
                     finaldb_dummy_period2[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns", 
                                    "GOLD LN Returns", 
                                    "MOM", 
                                    "3MTBILL LN Returns"]]).fit()

print(results_OLS_2_A6.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.8786
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.544
Time:                        14:41:26   Log-Likelihood:                 728.25
No. Observations:                 563   AIC:                            -1436.
Df Residuals:                     553   BIC:                            -1393.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
intercept              0.0021      0

In [385]:
#Model 7
finaldb_dummy_period2 = finaldb_dummy_period2.dropna()
finaldb_dummy_period2["intercept"] = 1
results_OLS_2_A7 = sm.OLS(finaldb_dummy_period2["BTC LN Returns"], 
                     finaldb_dummy_period2[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns", 
                                    "GOLD LN Returns", 
                                    "MOM", 
                                    "3MTBILL LN Returns", 
                                    "EX LN Returns"]]).fit()

print(results_OLS_2_A7.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.158
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.317
Time:                        14:41:21   Log-Likelihood:                 730.10
No. Observations:                 563   AIC:                            -1438.
Df Residuals:                     552   BIC:                            -1391.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
intercept              0.0007      0

In [384]:
#Model 8
finaldb_dummy_period2 = finaldb_dummy_period2.dropna()
finaldb_dummy_period2["intercept"] = 1
results_OLS_2_A8 = sm.OLS(finaldb_dummy_period2["BTC LN Returns"], 
                     finaldb_dummy_period2[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns", 
                                    "GOLD LN Returns", 
                                    "MOM", 
                                    "3MTBILL LN Returns", 
                                    "EX LN Returns", 
                                    "VIX LN Returns"]]).fit()

print(results_OLS_2_A8.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.098
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.361
Time:                        14:41:14   Log-Likelihood:                 730.35
No. Observations:                 563   AIC:                            -1437.
Df Residuals:                     551   BIC:                            -1385.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
intercept              0.0003      0

Regarding this second sub-sample period, we do not appreciate the same results as the authors. In fact, our key-findings are in-line regarding the negative correlation between Wednesday and BTC returns (still this is not significant). Moreover, the authors found a positive relationship that justifies significantly the day-of-the-week effect for Monday. We do not encounter the same conclusions. Nonetheless, we can state that BTC returns are significantly positively related with the Dollar Index ones, with a conclusion that may mean how in this sub-period BTC was considerably following the index and that hedging against a BTC downside by using EX was not possible.

### Period 2: Panel B
<a id="10"></a>
[Index](#contents)
- Second sub-sample to run OLS (January 2013-May 2015) with lag-values

In [356]:
finaldb_dummy["BTC LN Returns -1"] = finaldb_dummy["BTC LN Returns"].shift(1)
finaldb_dummy["SPX LN Returns -1"] = finaldb_dummy["SPX LN Returns"].shift(1)
finaldb_dummy["GOLD LN Returns -1"] = finaldb_dummy["GOLD LN Returns"].shift(1)
finaldb_dummy["MOM -1"] = finaldb_dummy["MOM"].shift(1)
finaldb_dummy["3MTBILL LN Returns -1"] = finaldb_dummy["3MTBILL LN Returns"].shift(1)
finaldb_dummy["EX LN Returns -1"] = finaldb_dummy["EX LN Returns"].shift(1)
finaldb_dummy["VIX LN Returns -1"] = finaldb_dummy["VIX LN Returns"].shift(1)
finaldb_dummy

,BTC LN Returns,EX LN Returns,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns,MOM,Day-of-the-week,M,T,W,TH,BTC LN Returns -1,intercept,SPX LN Returns -1,GOLD LN Returns -1,MOM -1,3MTBILL LN Returns -1,EX LN Returns -1,VIX LN Returns -1
2010-10-04,0.0038,0.0017,-0.0081,-0.0023,0.0448,-0.002,0.40,0,1,0,0,0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-05,0.0017,-0.0043,0.0206,0.0196,-0.0782,-0.036,0.24,1,0,1,0,0,0.0038,1,-0.0081,-0.0023,0.40,-0.002,0.0017,0.0448
2010-10-06,0.0227,-0.0043,-0.0007,0.0061,-0.0125,0.003,-0.62,2,0,0,1,0,0.0017,1,0.0206,0.0196,0.24,-0.036,-0.0043,-0.0782
2010-10-07,0.0646,0.0013,-0.0016,-0.0117,0.0033,0.000,0.24,3,0,0,0,1,0.0227,1,-0.0007,0.0061,-0.62,0.003,-0.0043,-0.0125
2010-10-08,0.2315,-0.0018,0.0061,0.0102,-0.0402,0.000,-0.04,4,0,0,0,0,0.0646,1,-0.0016,-0.0117,0.24,0.000,0.0013,0.0033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-10-26,0.0282,0.0020,0.0013,-0.0081,0.0062,-0.003,0.34,3,0,0,0,1,0.0388,1,-0.0047,0.0005,-0.10,-0.015,-0.0001,0.0063
2017-10-27,-0.0220,0.0043,0.0080,0.0046,-0.1424,-0.008,1.02,4,0,0,0,0,0.0282,1,0.0013,-0.0081,0.34,-0.003,0.0020,0.0062
2017-10-30,-0.0090,-0.0020,-0.0032,0.0026,0.0690,0.003,-0.17,0,1,0,0,0,-0.0220,1,0.0080,0.0046,1.02,-0.008,0.0043,-0.1424
2017-10-31,0.0493,-0.0010,0.0009,-0.0037,-0.0310,0.051,0.42,1,0,1,0,0,-0.0090,1,-0.0032,0.0026,-0.17,0.003,-0.0020,0.0690


In [358]:
#Period 2: PANEL B 
#truncate dataset to create second sub-sample 
finaldb_dummy_period2_B = finaldb_dummy.truncate(before=pd.Timestamp('2013-01-01') , after=pd.Timestamp('2015-05-31'))
finaldb_dummy_period2_B.head()

,BTC LN Returns,EX LN Returns,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns,MOM,Day-of-the-week,M,T,W,TH,BTC LN Returns -1,intercept,SPX LN Returns -1,GOLD LN Returns -1,MOM -1,3MTBILL LN Returns -1,EX LN Returns -1,VIX LN Returns -1
2013-01-03,0.0078,0.0011,-0.0021,-0.0140,-0.0082,-0.0051,-0.45,3,0,0,0,1,0.0074,1,0.0168,0.0117,0.24,0.0355,0.0005,-0.2318
2013-01-04,0.0050,0.0031,0.0049,-0.0040,-0.0514,0.0000,-0.36,4,0,0,0,0,0.0078,1,-0.0021,-0.0140,-0.45,-0.0051,0.0011,-0.0082
2013-01-07,0.0038,-0.0012,-0.0031,-0.0058,-0.0029,-0.0026,0.15,0,1,0,0,0,0.0050,1,0.0049,-0.0040,-0.36,0.0000,0.0031,-0.0514
2013-01-08,0.0164,0.0007,-0.0032,0.0072,-0.0124,-0.0025,0.86,1,0,1,0,0,0.0038,1,-0.0031,-0.0058,0.15,-0.0026,-0.0012,-0.0029
2013-01-09,0.0029,-0.0005,0.0027,-0.0008,0.0139,-0.0051,0.08,2,0,0,1,0,0.0164,1,-0.0032,0.0072,0.86,-0.0025,0.0007,-0.0124


In [359]:
#Model 1
finaldb_dummy_period2_B = finaldb_dummy_period2_B.dropna()
finaldb_dummy_period2_B["intercept"] = 1
results_OLS_1_B1 = sm.OLS(finaldb_dummy_period2_B["BTC LN Returns"], 
                     finaldb_dummy_period2_B[["intercept", "M", "T", "W", "TH"]]).fit()

print(results_OLS_1_B1.summary())

#Not significant

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9955
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.409
Time:                        14:27:05   Log-Likelihood:                 726.25
No. Observations:                 563   AIC:                            -1443.
Df Residuals:                     558   BIC:                            -1421.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0019      0.006      0.311      0.7

In [360]:
#Model 2
finaldb_dummy_period2_B = finaldb_dummy_period2_B.dropna()
finaldb_dummy_period2_B["intercept"] = 1
results_OLS_1_B2 = sm.OLS(finaldb_dummy_period2_B["BTC LN Returns"], 
                     finaldb_dummy_period2_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1"]]).fit()

print(results_OLS_1_B2.summary())

#Not significant

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.7950
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.554
Time:                        14:27:18   Log-Likelihood:                 726.25
No. Observations:                 563   AIC:                            -1441.
Df Residuals:                     557   BIC:                            -1415.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0019      0.00

In [361]:
#Model 3
finaldb_dummy_period2_B = finaldb_dummy_period2_B.dropna()
finaldb_dummy_period2_B["intercept"] = 1
results_OLS_1_B3 = sm.OLS(finaldb_dummy_period2_B["BTC LN Returns"], 
                     finaldb_dummy_period2_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1"]]).fit()

print(results_OLS_1_B3.summary())

#Not significant

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.7469
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.612
Time:                        14:27:23   Log-Likelihood:                 726.51
No. Observations:                 563   AIC:                            -1439.
Df Residuals:                     556   BIC:                            -1409.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0017      0.00

In [383]:
#Model 4
finaldb_dummy_period2_B = finaldb_dummy_period2_B.dropna()
finaldb_dummy_period2_B["intercept"] = 1
results_OLS_1_B4 = sm.OLS(finaldb_dummy_period2_B["BTC LN Returns"], 
                     finaldb_dummy_period2_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns -1"]]).fit()

print(results_OLS_1_B4.summary())

#Not significant

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.483
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.171
Time:                        14:41:02   Log-Likelihood:                 729.47
No. Observations:                 563   AIC:                            -1443.
Df Residuals:                     555   BIC:                            -1408.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
intercept              0.0019      0

In [382]:
#Model 5
finaldb_dummy_period2_B = finaldb_dummy_period2_B.dropna()
finaldb_dummy_period2_B["intercept"] = 1
results_OLS_1_B5 = sm.OLS(finaldb_dummy_period2_B["BTC LN Returns"], 
                     finaldb_dummy_period2_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns -1", 
                                    "MOM -1"]]).fit()

print(results_OLS_1_B5.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.433
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.180
Time:                        14:40:02   Log-Likelihood:                 730.02
No. Observations:                 563   AIC:                            -1442.
Df Residuals:                     554   BIC:                            -1403.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
intercept              0.0021      0

In [381]:
#Model 6
finaldb_dummy_period2_B = finaldb_dummy_period2_B.dropna()
finaldb_dummy_period2_B["intercept"] = 1
results_OLS_1_B6 = sm.OLS(finaldb_dummy_period2_B["BTC LN Returns"], 
                     finaldb_dummy_period2_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns -1", 
                                    "MOM -1", 
                                    "3MTBILL LN Returns -1"]]).fit()

print(results_OLS_1_B6.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.655
Date:                Sun, 24 Nov 2019   Prob (F-statistic):             0.0968
Time:                        14:39:49   Log-Likelihood:                 731.74
No. Observations:                 563   AIC:                            -1443.
Df Residuals:                     553   BIC:                            -1400.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
intercept                 0.00

In [380]:
#Model 7
finaldb_dummy_period2_B = finaldb_dummy_period2_B.dropna()
finaldb_dummy_period2_B["intercept"] = 1
results_OLS_1_B7 = sm.OLS(finaldb_dummy_period2_B["BTC LN Returns"], 
                     finaldb_dummy_period2_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns -1", 
                                    "MOM -1", 
                                    "3MTBILL LN Returns -1", 
                                    "EX LN Returns -1"]]).fit()

print(results_OLS_1_B7.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.797
Date:                Sun, 24 Nov 2019   Prob (F-statistic):             0.0582
Time:                        14:39:26   Log-Likelihood:                 733.27
No. Observations:                 563   AIC:                            -1445.
Df Residuals:                     552   BIC:                            -1397.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
intercept                 0.00

In [379]:
#Model 8
finaldb_dummy_period2_B = finaldb_dummy_period2_B.dropna()
finaldb_dummy_period2_B["intercept"] = 1
results_OLS_1_B8 = sm.OLS(finaldb_dummy_period2_B["BTC LN Returns"], 
                     finaldb_dummy_period2_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns -1", 
                                    "MOM -1", 
                                    "3MTBILL LN Returns -1", 
                                    "EX LN Returns -1", 
                                    "VIX LN Returns -1"]]).fit()

print(results_OLS_1_B8.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     1.828
Date:                Sun, 24 Nov 2019   Prob (F-statistic):             0.0466
Time:                        14:39:08   Log-Likelihood:                 734.34
No. Observations:                 563   AIC:                            -1445.
Df Residuals:                     551   BIC:                            -1393.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
intercept                 0.00

The results confirm what has been previously written about the significant positive relationship betwenn BTC returns and EX ones (in this case log-EX returns). This relationship is stated by the Authors as well: "There are hints of other relationships with the S&P500 and EX, but they are partial and inconsistent". Another interesting relationship (positive) is the one with GOLD, that thefore cannot be used to hedge a potential portfolio against an unwanted fluctuation in BTC. 

### Period 3: Panel A
<a id="11"></a>
[Index](#contents)
- Third sub-sample to run OLS (June 2015-October 2017) 

In [368]:
#Period 3: PANEL A
finaldb_dummy

,BTC LN Returns,EX LN Returns,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns,MOM,Day-of-the-week,M,T,W,TH,BTC LN Returns -1,intercept,SPX LN Returns -1,GOLD LN Returns -1,MOM -1,3MTBILL LN Returns -1,EX LN Returns -1,VIX LN Returns -1
2010-10-04,0.0038,0.0017,-0.0081,-0.0023,0.0448,-0.002,0.40,0,1,0,0,0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-05,0.0017,-0.0043,0.0206,0.0196,-0.0782,-0.036,0.24,1,0,1,0,0,0.0038,1,-0.0081,-0.0023,0.40,-0.002,0.0017,0.0448
2010-10-06,0.0227,-0.0043,-0.0007,0.0061,-0.0125,0.003,-0.62,2,0,0,1,0,0.0017,1,0.0206,0.0196,0.24,-0.036,-0.0043,-0.0782
2010-10-07,0.0646,0.0013,-0.0016,-0.0117,0.0033,0.000,0.24,3,0,0,0,1,0.0227,1,-0.0007,0.0061,-0.62,0.003,-0.0043,-0.0125
2010-10-08,0.2315,-0.0018,0.0061,0.0102,-0.0402,0.000,-0.04,4,0,0,0,0,0.0646,1,-0.0016,-0.0117,0.24,0.000,0.0013,0.0033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-10-26,0.0282,0.0020,0.0013,-0.0081,0.0062,-0.003,0.34,3,0,0,0,1,0.0388,1,-0.0047,0.0005,-0.10,-0.015,-0.0001,0.0063
2017-10-27,-0.0220,0.0043,0.0080,0.0046,-0.1424,-0.008,1.02,4,0,0,0,0,0.0282,1,0.0013,-0.0081,0.34,-0.003,0.0020,0.0062
2017-10-30,-0.0090,-0.0020,-0.0032,0.0026,0.0690,0.003,-0.17,0,1,0,0,0,-0.0220,1,0.0080,0.0046,1.02,-0.008,0.0043,-0.1424
2017-10-31,0.0493,-0.0010,0.0009,-0.0037,-0.0310,0.051,0.42,1,0,1,0,0,-0.0090,1,-0.0032,0.0026,-0.17,0.003,-0.0020,0.0690


In [370]:
finaldb_dummy_period3 = finaldb_dummy.truncate(before=pd.Timestamp('2015-06-01') , after=pd.Timestamp('2017-10-31'))
finaldb_dummy_period3.tail()

,BTC LN Returns,EX LN Returns,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns,MOM,Day-of-the-week,M,T,W,TH,BTC LN Returns -1,intercept,SPX LN Returns -1,GOLD LN Returns -1,MOM -1,3MTBILL LN Returns -1,EX LN Returns -1,VIX LN Returns -1
2017-10-25,0.0388,-0.0001,-0.0047,0.0005,0.0063,-0.015,-0.10,2,0,0,1,0,-0.0650,1,0.0016,-0.0042,0.69,0.028,0.0015,0.0081
2017-10-26,0.0282,0.0020,0.0013,-0.0081,0.0062,-0.003,0.34,3,0,0,0,1,0.0388,1,-0.0047,0.0005,-0.10,-0.015,-0.0001,0.0063
2017-10-27,-0.0220,0.0043,0.0080,0.0046,-0.1424,-0.008,1.02,4,0,0,0,0,0.0282,1,0.0013,-0.0081,0.34,-0.003,0.0020,0.0062
2017-10-30,-0.0090,-0.0020,-0.0032,0.0026,0.0690,0.003,-0.17,0,1,0,0,0,-0.0220,1,0.0080,0.0046,1.02,-0.008,0.0043,-0.1424
2017-10-31,0.0493,-0.0010,0.0009,-0.0037,-0.0310,0.051,0.42,1,0,1,0,0,-0.0090,1,-0.0032,0.0026,-0.17,0.003,-0.0020,0.0690


In [371]:
#Model 1
finaldb_dummy_period3 = finaldb_dummy_period3.dropna()
finaldb_dummy_period3["intercept"] = 1
results_OLS_3_A1 = sm.OLS(finaldb_dummy_period3["BTC LN Returns"], 
                     finaldb_dummy_period3[["intercept", "M", "T", "W", "TH"]]).fit()

print(results_OLS_3_A1.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.4380
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.781
Time:                        14:37:09   Log-Likelihood:                 1083.1
No. Observations:                 576   AIC:                            -2156.
Df Residuals:                     571   BIC:                            -2134.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0027      0.003      0.790      0.4

In [372]:
#Model 2
finaldb_dummy_period3 = finaldb_dummy_period3.dropna()
finaldb_dummy_period3["intercept"] = 1
results_OLS_3_A2 = sm.OLS(finaldb_dummy_period3["BTC LN Returns"], 
                     finaldb_dummy_period3[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1"]]).fit()

print(results_OLS_3_A2.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.3502
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.882
Time:                        14:37:29   Log-Likelihood:                 1083.1
No. Observations:                 576   AIC:                            -2154.
Df Residuals:                     570   BIC:                            -2128.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0027      0.00

In [395]:
#Model 3
finaldb_dummy_period3 = finaldb_dummy_period3.dropna()
finaldb_dummy_period3["intercept"] = 1
results_OLS_3_A3 = sm.OLS(finaldb_dummy_period3["BTC LN Returns"], 
                     finaldb_dummy_period3[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns"]]).fit()

print(results_OLS_3_A3.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.2919
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.941
Time:                        14:44:03   Log-Likelihood:                 1083.1
No. Observations:                 576   AIC:                            -2152.
Df Residuals:                     569   BIC:                            -2122.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0027      0.00

In [396]:
#Model 4
finaldb_dummy_period3 = finaldb_dummy_period3.dropna()
finaldb_dummy_period3["intercept"] = 1
results_OLS_3_A4 = sm.OLS(finaldb_dummy_period3["BTC LN Returns"], 
                     finaldb_dummy_period3[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns", 
                                    "GOLD LN Returns"]]).fit()

print(results_OLS_3_A4.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.5274
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.814
Time:                        14:44:06   Log-Likelihood:                 1084.1
No. Observations:                 576   AIC:                            -2152.
Df Residuals:                     568   BIC:                            -2117.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0023      0.00

In [397]:
#Model 5
finaldb_dummy_period3 = finaldb_dummy_period3.dropna()
finaldb_dummy_period3["intercept"] = 1
results_OLS_3_A5 = sm.OLS(finaldb_dummy_period3["BTC LN Returns"], 
                     finaldb_dummy_period3[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns", 
                                    "GOLD LN Returns", 
                                    "MOM"]]).fit()

print(results_OLS_3_A5.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.5685
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.804
Time:                        14:44:09   Log-Likelihood:                 1084.5
No. Observations:                 576   AIC:                            -2151.
Df Residuals:                     567   BIC:                            -2112.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0023      0.00

In [398]:
#Model 6
finaldb_dummy_period3 = finaldb_dummy_period3.dropna()
finaldb_dummy_period3["intercept"] = 1
results_OLS_3_A6 = sm.OLS(finaldb_dummy_period3["BTC LN Returns"], 
                     finaldb_dummy_period3[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns", 
                                    "MOM", 
                                    "3MTBILL LN Returns"]]).fit()

print(results_OLS_3_A6.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.6203
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.780
Time:                        14:44:25   Log-Likelihood:                 1085.0
No. Observations:                 576   AIC:                            -2150.
Df Residuals:                     566   BIC:                            -2106.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
intercept              0.0021      0

In [399]:
#Model 7
finaldb_dummy_period3 = finaldb_dummy_period3.dropna()
finaldb_dummy_period3["intercept"] = 1
results_OLS_3_A7 = sm.OLS(finaldb_dummy_period3["BTC LN Returns"], 
                     finaldb_dummy_period3[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns", 
                                    "MOM", 
                                    "3MTBILL LN Returns", 
                                    "EX LN Returns"]]).fit()

print(results_OLS_3_A7.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.7116
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.714
Time:                        14:44:52   Log-Likelihood:                 1085.8
No. Observations:                 576   AIC:                            -2150.
Df Residuals:                     565   BIC:                            -2102.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
intercept              0.0018      0

In [400]:
#Model 8
finaldb_dummy_period3 = finaldb_dummy_period3.dropna()
finaldb_dummy_period3["intercept"] = 1
results_OLS_3_A8 = sm.OLS(finaldb_dummy_period3["BTC LN Returns"], 
                     finaldb_dummy_period3[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns", 
                                    "MOM", 
                                    "3MTBILL LN Returns", 
                                    "EX LN Returns", 
                                    "VIX LN Returns"]]).fit()

print(results_OLS_3_A8.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.6542
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.782
Time:                        14:44:59   Log-Likelihood:                 1085.9
No. Observations:                 576   AIC:                            -2148.
Df Residuals:                     564   BIC:                            -2095.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
intercept              0.0018      0

The P-value are not significant to underline a day-of-the-week effect. Interesting enough anyway, the only positive one would have been monday and Thursday, with a significant negative relationship in tuesday

### Period 3: Panel B
<a id="12"></a>
[Index](#contents)
- Third sub-sample to run OLS (June 2015-October 2017) with lag-values

In [403]:
finaldb_dummy["BTC LN Returns -1"] = finaldb_dummy["BTC LN Returns"].shift(1)
finaldb_dummy["SPX LN Returns -1"] = finaldb_dummy["SPX LN Returns"].shift(1)
finaldb_dummy["GOLD LN Returns -1"] = finaldb_dummy["GOLD LN Returns"].shift(1)
finaldb_dummy["MOM -1"] = finaldb_dummy["MOM"].shift(1)
finaldb_dummy["3MTBILL LN Returns -1"] = finaldb_dummy["3MTBILL LN Returns"].shift(1)
finaldb_dummy["EX LN Returns -1"] = finaldb_dummy["EX LN Returns"].shift(1)
finaldb_dummy["VIX LN Returns -1"] = finaldb_dummy["VIX LN Returns"].shift(1)
finaldb_dummy

,BTC LN Returns,EX LN Returns,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns,MOM,Day-of-the-week,M,T,W,TH,BTC LN Returns -1,intercept,SPX LN Returns -1,GOLD LN Returns -1,MOM -1,3MTBILL LN Returns -1,EX LN Returns -1,VIX LN Returns -1
2010-10-04,0.0038,0.0017,-0.0081,-0.0023,0.0448,-0.002,0.40,0,1,0,0,0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-05,0.0017,-0.0043,0.0206,0.0196,-0.0782,-0.036,0.24,1,0,1,0,0,0.0038,1,-0.0081,-0.0023,0.40,-0.002,0.0017,0.0448
2010-10-06,0.0227,-0.0043,-0.0007,0.0061,-0.0125,0.003,-0.62,2,0,0,1,0,0.0017,1,0.0206,0.0196,0.24,-0.036,-0.0043,-0.0782
2010-10-07,0.0646,0.0013,-0.0016,-0.0117,0.0033,0.000,0.24,3,0,0,0,1,0.0227,1,-0.0007,0.0061,-0.62,0.003,-0.0043,-0.0125
2010-10-08,0.2315,-0.0018,0.0061,0.0102,-0.0402,0.000,-0.04,4,0,0,0,0,0.0646,1,-0.0016,-0.0117,0.24,0.000,0.0013,0.0033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-10-26,0.0282,0.0020,0.0013,-0.0081,0.0062,-0.003,0.34,3,0,0,0,1,0.0388,1,-0.0047,0.0005,-0.10,-0.015,-0.0001,0.0063
2017-10-27,-0.0220,0.0043,0.0080,0.0046,-0.1424,-0.008,1.02,4,0,0,0,0,0.0282,1,0.0013,-0.0081,0.34,-0.003,0.0020,0.0062
2017-10-30,-0.0090,-0.0020,-0.0032,0.0026,0.0690,0.003,-0.17,0,1,0,0,0,-0.0220,1,0.0080,0.0046,1.02,-0.008,0.0043,-0.1424
2017-10-31,0.0493,-0.0010,0.0009,-0.0037,-0.0310,0.051,0.42,1,0,1,0,0,-0.0090,1,-0.0032,0.0026,-0.17,0.003,-0.0020,0.0690


In [405]:
#Period 3: PANEL B 
#truncate dataset to create second sub-sample 
finaldb_dummy_period3_B = finaldb_dummy.truncate(before=pd.Timestamp('2015-06-01') , after=pd.Timestamp('2017-10-31'))
finaldb_dummy_period3_B.tail()

,BTC LN Returns,EX LN Returns,SPX LN Returns,GOLD LN Returns,VIX LN Returns,3MTBILL LN Returns,MOM,Day-of-the-week,M,T,W,TH,BTC LN Returns -1,intercept,SPX LN Returns -1,GOLD LN Returns -1,MOM -1,3MTBILL LN Returns -1,EX LN Returns -1,VIX LN Returns -1
2017-10-25,0.0388,-0.0001,-0.0047,0.0005,0.0063,-0.015,-0.10,2,0,0,1,0,-0.0650,1,0.0016,-0.0042,0.69,0.028,0.0015,0.0081
2017-10-26,0.0282,0.0020,0.0013,-0.0081,0.0062,-0.003,0.34,3,0,0,0,1,0.0388,1,-0.0047,0.0005,-0.10,-0.015,-0.0001,0.0063
2017-10-27,-0.0220,0.0043,0.0080,0.0046,-0.1424,-0.008,1.02,4,0,0,0,0,0.0282,1,0.0013,-0.0081,0.34,-0.003,0.0020,0.0062
2017-10-30,-0.0090,-0.0020,-0.0032,0.0026,0.0690,0.003,-0.17,0,1,0,0,0,-0.0220,1,0.0080,0.0046,1.02,-0.008,0.0043,-0.1424
2017-10-31,0.0493,-0.0010,0.0009,-0.0037,-0.0310,0.051,0.42,1,0,1,0,0,-0.0090,1,-0.0032,0.0026,-0.17,0.003,-0.0020,0.0690


In [407]:
#Model 1
finaldb_dummy_period3_B = finaldb_dummy_period3_B.dropna()
finaldb_dummy_period3_B["intercept"] = 1
results_OLS_3_B1 = sm.OLS(finaldb_dummy_period3_B["BTC LN Returns"], 
                     finaldb_dummy_period3_B[["intercept", "M", "T", "W", "TH"]]).fit()

print(results_OLS_3_B1.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.4380
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.781
Time:                        14:47:10   Log-Likelihood:                 1083.1
No. Observations:                 576   AIC:                            -2156.
Df Residuals:                     571   BIC:                            -2134.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0027      0.003      0.790      0.4

In [408]:
#Model 2
finaldb_dummy_period3_B = finaldb_dummy_period3_B.dropna()
finaldb_dummy_period3_B["intercept"] = 1
results_OLS_3_B2 = sm.OLS(finaldb_dummy_period3_B["BTC LN Returns"], 
                     finaldb_dummy_period3_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1"]]).fit()

print(results_OLS_3_B2.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.3502
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.882
Time:                        14:47:23   Log-Likelihood:                 1083.1
No. Observations:                 576   AIC:                            -2154.
Df Residuals:                     570   BIC:                            -2128.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0027      0.00

In [409]:
#Model 3
finaldb_dummy_period3_B = finaldb_dummy_period3_B.dropna()
finaldb_dummy_period3_B["intercept"] = 1
results_OLS_3_B3 = sm.OLS(finaldb_dummy_period3_B["BTC LN Returns"], 
                     finaldb_dummy_period3_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1"]]).fit()

print(results_OLS_3_B3.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.2936
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.940
Time:                        14:47:44   Log-Likelihood:                 1083.1
No. Observations:                 576   AIC:                            -2152.
Df Residuals:                     569   BIC:                            -2122.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0027      0.00

In [410]:
#Model 4
finaldb_dummy_period3_B = finaldb_dummy_period3_B.dropna()
finaldb_dummy_period3_B["intercept"] = 1
results_OLS_3_B4 = sm.OLS(finaldb_dummy_period3_B["BTC LN Returns"], 
                     finaldb_dummy_period3_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns -1"]]).fit()

print(results_OLS_3_B4.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                    0.2948
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.956
Time:                        14:47:54   Log-Likelihood:                 1083.2
No. Observations:                 576   AIC:                            -2150.
Df Residuals:                     568   BIC:                            -2116.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
intercept              0.0026      0

In [411]:
#Model 5
finaldb_dummy_period3_B = finaldb_dummy_period3_B.dropna()
finaldb_dummy_period3_B["intercept"] = 1
results_OLS_3_B5 = sm.OLS(finaldb_dummy_period3_B["BTC LN Returns"], 
                     finaldb_dummy_period3_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns -1", 
                                    "MOM -1"]]).fit()

print(results_OLS_3_B5.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                    0.2680
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.976
Time:                        14:48:13   Log-Likelihood:                 1083.3
No. Observations:                 576   AIC:                            -2149.
Df Residuals:                     567   BIC:                            -2109.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
intercept              0.0026      0

In [412]:
#Model 6
finaldb_dummy_period3_B = finaldb_dummy_period3_B.dropna()
finaldb_dummy_period3_B["intercept"] = 1
results_OLS_3_B6 = sm.OLS(finaldb_dummy_period3_B["BTC LN Returns"], 
                     finaldb_dummy_period3_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns -1", 
                                    "MOM -1", 
                                    "3MTBILL LN Returns -1"]]).fit()

print(results_OLS_3_B6.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                    0.3458
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.959
Time:                        14:48:30   Log-Likelihood:                 1083.8
No. Observations:                 576   AIC:                            -2148.
Df Residuals:                     566   BIC:                            -2104.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
intercept                 0.00

In [413]:
#Model 7
finaldb_dummy_period3_B = finaldb_dummy_period3_B.dropna()
finaldb_dummy_period3_B["intercept"] = 1
results_OLS_3_B7 = sm.OLS(finaldb_dummy_period3_B["BTC LN Returns"], 
                     finaldb_dummy_period3_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns -1", 
                                    "MOM -1", 
                                    "3MTBILL LN Returns -1", 
                                    "EX LN Returns -1"]]).fit()

print(results_OLS_3_B7.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                    0.3443
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.969
Time:                        14:48:40   Log-Likelihood:                 1083.9
No. Observations:                 576   AIC:                            -2146.
Df Residuals:                     565   BIC:                            -2098.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
intercept                 0.00

In [414]:
#Model 8
finaldb_dummy_period3_B = finaldb_dummy_period3_B.dropna()
finaldb_dummy_period3_B["intercept"] = 1
results_OLS_3_B8 = sm.OLS(finaldb_dummy_period3_B["BTC LN Returns"], 
                     finaldb_dummy_period3_B[["intercept", "M", "T", "W", "TH", 
                                    "BTC LN Returns -1", 
                                    "SPX LN Returns -1", 
                                    "GOLD LN Returns -1", 
                                    "MOM -1", 
                                    "3MTBILL LN Returns -1", 
                                    "EX LN Returns -1", 
                                    "VIX LN Returns -1"]]).fit()

print(results_OLS_3_B8.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.011
Method:                 Least Squares   F-statistic:                    0.4078
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.953
Time:                        14:48:52   Log-Likelihood:                 1084.5
No. Observations:                 576   AIC:                            -2145.
Df Residuals:                     564   BIC:                            -2093.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
intercept                 0.00

Even with lagged-values, we do not appreciate a significant day-of-the-week relationship 

In [269]:
#Conclusive regression over the entire time-lag 
finaldb_dummy = finaldb_dummy.dropna()
finaldb_dummy["intercept"] = 1
results_OLS = sm.OLS(finaldb_dummy["BTC LN Returns"], finaldb_dummy[["intercept", "M", "T", "W", "TH"]]).fit()

print(results_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:         BTC LN Returns   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.5170
Date:                Sun, 24 Nov 2019   Prob (F-statistic):              0.723
Time:                        12:39:54   Log-Likelihood:                 2320.9
No. Observations:                1676   AIC:                            -4632.
Df Residuals:                    1671   BIC:                            -4605.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0034      0.003      1.047      0.2

A further research in the field could focus (as the author did) in modeling ARCH and GARCH. GARCH (Generalized AutoRegressive Conditional Heteroskedasticity) estimates volatility instead of returns. It is based on the assumptions that variables are stationary and that conditional volatility is time-dependent, thus it is possible to model volatility clustering. It is often used when variance error is believed to be serially autocorrelated, its application is recommended for time series with high frequency (i.e. from daily to monthly data), because the smaller the number of data, the larger the approximation error. This model has been implemented in order to study financial and macroeconomic phenomena.

[Back to Index](#contents)